In [3]:
%load_ext autoreload
%autoreload 2
    
import spikeinterface.sorters as ss
from dandi.dandiapi import DandiAPIClient
import spikeinterface.extractors as se

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import torch
print(torch.cuda.is_available())  # check if CUDA is available
print(torch.cuda.get_device_name(0))  # get the name of the GPU

In [6]:
%%time
# load RecordingExtractor

# load dandiset (npx, evoked, 20Khz)
dandiset_id = '001250'
filepath = 'sub-002-fitted/sub-002-fitted_ecephys.nwb'

# get the file path on S3
with DandiAPIClient() as client:
    asset = client.get_dandiset(dandiset_id, 'draft').get_asset_by_path(filepath)
    s3_path = asset.get_content_url(follow_redirects=1, strip_query=True)
print("s3_path:", s3_path)

# get RecordingExtractor
Recording = se.NwbRecordingExtractor(file_path=s3_path, stream_mode="remfile")

# report
print('\n', Recording)

# spike sort
sorting_KS3 = ss.run_sorter(sorter_name="kilosort3", singularity_image=True, recording=Recording)